<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<img src="mioti.png" style="height: 80px">

<center style="color:#888">Data Science with Python</center>

# DSPy5 Pandas "advanced". Challenge Solutions.

<img src="madrid_atasco.jpg" style="width:800px">

**Inicialización**

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = (10, 5)

Vamos a trabajar con el dataset del número de accidentes de tráfico en Madrid durante el 2019 (fuente: [https://datos.madrid.es/](https://datos.madrid.es/)). Fichero explicativo: `Estructura_DS_Accidentes_trafico_desde_2019.pdf`

In [ ]:
df=pd.read_excel('2019_Accidentalidad.xlsx')

In [ ]:
df

**Ejercicio 1:** ¿Quién está implicado en más accidentes de tráfico, los hombres o las mujeres? ¿Y en función del rango de edad? ¿Hay alguna franja de edad en la que las mujeres esten implicadas en más accidentes que los hombres?

In [ ]:
df['SEXO'].value_counts(dropna=False)

In [ ]:
df.groupby('SEXO').size()

In [ ]:
df['RANGO EDAD'].value_counts(dropna=False)

In [ ]:
df.groupby('RANGO EDAD').size().sort_values(ascending=False)

In [ ]:
pivot_edad_sexo=df.pivot_table(values='Nº  EXPEDIENTE',index='RANGO EDAD',columns='SEXO',aggfunc='count')
pivot_edad_sexo

In [ ]:
df.groupby(['RANGO EDAD','SEXO']).agg({'Nº  EXPEDIENTE':'count'})

In [ ]:
pivot_edad_sexo[pivot_edad_sexo['Hombre']<pivot_edad_sexo['Mujer']]

In [ ]:
franja_edad=pivot_edad_sexo[pivot_edad_sexo['Hombre']<pivot_edad_sexo['Mujer']].index[0]
franja_edad

**Ejercicio 2:** Para el rango de edad para el cual hemos obtenido que hay más accidentes de tráfico con mujeres implicadas que con hombres, ¿esas mujeres eran conductoras, peatones o pasajeras? Para los casos en los que era conductora, ¿había alguna saltándose la ley y conduciendo un vehículo para el que necesitas carnet?

In [ ]:
df_mujer_10_14=df.loc[(df['SEXO']=='Mujer')&(df['RANGO EDAD']==franja_edad),]
df_mujer_10_14

In [ ]:
df_mujer_10_14['TIPO PERSONA'].value_counts()

In [ ]:
df_mujer_10_14.loc[df_mujer_10_14['TIPO PERSONA']=='Conductor','TIPO VEHÍCULO'].value_counts()

A modo curiosidad:

In [ ]:
df_menor_edad_conductor=df[(df['RANGO EDAD'].isin(['DE 0 A 5 AÑOS','DE 6 A 9 AÑOS','DE 10 A 14 AÑOS','DE 15 A 17 AÑOS']))
  &(df['TIPO PERSONA']=='Conductor')]
df_menor_edad_conductor

In [ ]:
df_menor_edad_conductor.pivot_table(values='Nº  EXPEDIENTE',index='SEXO',columns='TIPO VEHÍCULO',aggfunc='count')

**Ejercicio 3:** ¿Cuál fue el día del año en el que hubo más accidentes? Haz un plot del nº de accidentes por mes del año ¿Es el número de accidentes constante a lo largo de los meses de 2019?. _Hint: mismo Nº EXPEDIENTE implica que se trata del mismo accidente donde hay varios implicados y por lo tanto solo hay que contarlo una vez.

Una posible opción es quedarnos con el número de expediente y la fecha y eliminar los duplicados:

In [ ]:
df_accidente_fecha=df[['Nº  EXPEDIENTE','FECHA']].drop_duplicates()
#Igual a:
#df_accidente_fecha=df.drop_duplicates(subset=['Nº  EXPEDIENTE'])
df_accidente_fecha

In [ ]:
df_accidente_fecha.groupby('FECHA').size().sort_values(ascending=False)

In [ ]:
df_5_abril=df[df['FECHA']=='05/04/2019'][['Nº  EXPEDIENTE','ESTADO METEREOLÓGICO']].drop_duplicates()
df_5_abril

In [ ]:
df_5_abril['ESTADO METEREOLÓGICO'].value_counts()

Para acceder al mes, hay varias opciones: una es pasarlo a datetime (aunque lo veremos en la siguiente sesión)  y otra opción es usar las operaciones vectorizadas con string de pandas:

In [ ]:
df_accidente_fecha

In [ ]:
df_accidente_fecha['Mes'] = df_accidente_fecha['FECHA'].str[3:5]
#o:
#df_accidente_fecha['Mes'] = df_accidente_fecha['FECHA'].str.split('/').str[1]

In [ ]:
df_accidente_fecha

In [ ]:
df_accidente_fecha.groupby('Mes').size()

In [ ]:
df_accidente_fecha.groupby('Mes').size().plot(title="Nº de accidentes por mes en Madrid", figsize=(10, 5), color='b')

**Ejercicio 4:** ¿Cuál fue el distrito con más accidentes? ¿y la calle? _Hint: de nuevo recordar que estamos hablando de nº de accidentes y el campo Nº EXPEDIENTE puede aparecer tantas veces como personas haya habido involucradas en el accidente.

In [ ]:
df_calle_distrito=df[['Nº  EXPEDIENTE','CALLE','DISTRITO']].drop_duplicates()
df_calle_distrito

In [ ]:
df_calle_distrito['DISTRITO'].value_counts(dropna=False)

In [ ]:
df_calle_distrito['CALLE'].value_counts(dropna=False)

**Ejercicio 5:** De las 7 calles con más accidentes del apartado anterior, genera una columna con la longitud de esa calle en metros o km (mismamente buscando en wikipedia) y obten el nº de accidentes por longitud de la calle. Una vez hecho esto podemos ver cual es la calle más propensa a tener accidentes de las 7 seleccionadas.

In [ ]:
serie_calles=df_calle_distrito['CALLE'].value_counts(dropna=False).head(7)
serie_calles

In [ ]:
df_calles=pd.DataFrame({'Accidentes':serie_calles})
df_calles

In [ ]:
df_calles.loc[:,'Longitud']=[6300,10500,4000,2500,1910,4200,11500]

In [ ]:
df_calles['Nº Accidentes por metro']=df_calles['Accidentes']/df_calles['Longitud']
df_calles.sort_values(by='Nº Accidentes por metro',ascending=False)

<img src="castellana.jpg" style="width:600px">

**Ejercicio 6:** Vamos a seleccionar los accidentes donde el tipo de vehículo implicado son las bicicletas. También disponemos del tráfico de bicicletas en Madrid durante el 2019 (BICICLETAS_2019.xlxs). Estudiar si existe algún tipo de correlación entre los distritos con más accidentes con bicicletas implicadas y los distritos donde más se usa la bicicleta (la comparativa puede ser a nivel nº total de accidentes con bicicleta en cada distrito en el año 2019 vs nº de bicicletas usadas por distrito en el año 2019).

In [ ]:
df_bicicletas_distritos=df.loc[df['TIPO VEHÍCULO']=='Bicicleta',['Nº  EXPEDIENTE','DISTRITO']].drop_duplicates()
df_bicicletas_distritos

In [ ]:
acc_bic_distr=df_bicicletas_distritos.groupby('DISTRITO').size()
acc_bic_distr=pd.DataFrame({'Accidentes':acc_bic_distr})
acc_bic_distr

In [ ]:
trafico_bicicletas=pd.read_excel('BICICLETAS_2019.xlsx')

In [ ]:
trafico_bicicletas

In [ ]:
trafico_bicicletas=trafico_bicicletas.groupby('DISTRITO').aggregate({'BICICLETAS':'sum'})
trafico_bicicletas

In [ ]:
trafico_bicicletas.index

Los distritos están en mayusculas y tienen un espacio delante en blanco:

In [ ]:
trafico_bicicletas.index=trafico_bicicletas.index.str[1:]

In [ ]:
trafico_bicicletas.index=trafico_bicicletas.index.str.upper()

In [ ]:
trafico_bicicletas

In [ ]:
accidentes_vs_n_bicicletas=pd.merge(left=acc_bic_distr,right=trafico_bicicletas, left_index=True, right_index=True)
accidentes_vs_n_bicicletas

In [ ]:
accidentes_vs_n_bicicletas['Indice']=accidentes_vs_n_bicicletas['Accidentes']/accidentes_vs_n_bicicletas['BICICLETAS']
accidentes_vs_n_bicicletas.sort_values(by='Indice',ascending=False)

In [ ]:
plt.plot(accidentes_vs_n_bicicletas['Accidentes'], accidentes_vs_n_bicicletas['BICICLETAS'], 'o', alpha=0.5)
plt.xlabel("Nº Accidentes")
plt.ylabel("Nº Bicicletas")

**Ejercicio 7:** Por último, tambien disponemos de los accidentes en 2020 (`2020_Accidentalidad.xlsx`). Comparar la serie mensual del nº de accidentes en 2019 vs 2020 y ver cuanto se han reducidos los accidentes cada mes debido al efecto del confinamiento.

In [ ]:
df_2020=pd.read_excel('2020_Accidentalidad.xlsx')
df_2020.head(5)

In [ ]:
df_accidente_fecha_2020=df_2020[['Nº  EXPEDIENTE','FECHA']].drop_duplicates()
df_accidente_fecha_2020['Mes'] = df_accidente_fecha_2020['FECHA'].str[3:5]
df_accidente_fecha_2020.groupby('Mes').size()

In [ ]:
ax=df_accidente_fecha.groupby('Mes').size().plot(title="Nº de accidentes por mes en Madrid", figsize=(10, 5), color='b',label='2019')
df_accidente_fecha_2020.groupby('Mes').size().plot(ax=ax, color='r',label='2020')
ax.legend()

In [ ]:
df_2019_2020=pd.DataFrame({'2019':df_accidente_fecha.groupby('Mes').size(),'2020':df_accidente_fecha_2020.groupby('Mes').size()})
df_2019_2020

In [ ]:
df_2019_2020['Variación']=df_2019_2020['2019']/df_2019_2020['2020']
df_2019_2020

In [ ]:
df_2021=pd.read_excel('2021_Accidentalidad.xlsx')
df_accidente_fecha_2021=df_2021[['Nº  EXPEDIENTE','FECHA']].drop_duplicates()
df_accidente_fecha_2021['Mes'] = df_accidente_fecha_2021['FECHA'].str[3:5]
df_accidente_fecha_2021.groupby('Mes').size()

In [ ]:
ax=df_accidente_fecha.groupby('Mes').size().plot(title="Nº de accidentes por mes en Madrid", figsize=(10, 5), color='b',label='2019')
df_accidente_fecha_2020.groupby('Mes').size().plot(ax=ax, color='r',label='2020')
df_accidente_fecha_2021.groupby('Mes').size().plot(ax=ax, color='orange',label='2021')
ax.legend()